## 낚시성 기사 분류

<br>

[낚시성 기사 탐지 데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=71338)
<br>[낚시성 기사 분류](https://aifactory.space/task/2663/overview)

<br>

<b>Load Data</b>
<br>구글 드라이브에 저장된 AIHUB Data를 Load하여 압축 해제

<br><b>Model Training</b>
<br>낚시성 기사 분류 Task에 대한 Training 진행

<br><b>Model Test</b>
<br>기사의 낚시성 예측값(0, 1)을 pred_y.csv에 저장

<br><b>Submission</b>
<br>pred_y.csv를 리더보드에 제출하여 채점 진행


<br>

### GPU Information

In [ ]:
!nvidia-smi

### Load Data

In [ ]:
!pip install gdown

In [ ]:
!pip install transformers
# !pip install transformers==4.29.2
# !pip install safetensors==0.3.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_id = "1q1nO2sUI8j_CgIhs79QIYBzhLCZ1O9En"
url = f"https://drive.google.com/uc?id={file_id}"

In [ ]:
import os

before_files = set(os.listdir())

!gdown {url}

after_files = set(os.listdir())

downloaded_files = after_files - before_files
if downloaded_files:
    filename = downloaded_files.pop()
    print(f"Downloaded file: {filename}")
    downloaded_filepath = filename
else:
    print("No file downloaded.")

Downloading...
From: https://drive.google.com/uc?id=1q1nO2sUI8j_CgIhs79QIYBzhLCZ1O9En
To: /content/aihub_clickbait_classification_data.zip
100% 57.3M/57.3M [00:00<00:00, 72.4MB/s]
Downloaded file: aihub_clickbait_classification_data.zip


In [ ]:
!unzip {downloaded_filepath}

Archive:  aihub_clickbait_classification_data.zip
   creating: aihub_clickbait_classification_data/
  inflating: aihub_clickbait_classification_data/test_x.csv  
  inflating: aihub_clickbait_classification_data/train.csv  


### Model Training

In [ ]:
import sys
import os
import datetime
import argparse
import pandas as pd
from tqdm import tqdm

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

from transformers import AutoTokenizer, RobertaForSequenceClassification, AdamW


def main(pretrained_model, train_data_path,  model_save_folder, epoch, batch_size):

    # os.mkdir(model_save_folder)

    class ClickbaitDetectionDataset(Dataset):

        def __init__(self, dataset):
            self.dataset = dataset
            self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

        def __len__(self):
            return len(self.dataset)

        def __getitem__(self, idx):
            row = self.dataset.iloc[idx, 0:2].values
            text = row[0]
            y = row[1]

            inputs = self.tokenizer(
                text,
                return_tensors='pt',
                truncation=True,
                max_length=256,
                pad_to_max_length=True,
                add_special_tokens=True
                )

            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]

            return input_ids, attention_mask, y

    if torch.cuda.is_available() == True:
        device = torch.device("cuda:0")
        model = RobertaForSequenceClassification.from_pretrained(pretrained_model).to(device)
    elif torch.cuda.is_available() == False:
        model = RobertaForSequenceClassification.from_pretrained(pretrained_model)

    train_data = pd.read_csv(train_data_path)
    train_dataset = ClickbaitDetectionDataset(train_data)

    optimizer = AdamW(model.parameters(), lr=1e-5)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    train_time = []
    train_loss = []
    train_accuracy = []

    for i in range(epoch):
        total_loss = 0.0
        correct = 0
        total = 0
        batches = 0

        model.train()

        with tqdm(train_loader) as pbar:
            pbar.set_description("Epoch " + str(i + 1))
            for input_ids_batch, attention_masks_batch, y_batch in pbar:
                optimizer.zero_grad()

                if torch.cuda.is_available() == True:
                    y_batch = y_batch.to(device)
                    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]

                elif torch.cuda.is_available() == False:
                    y_batch = y_batch
                    y_pred = model(input_ids_batch, attention_mask=attention_masks_batch)[0]


                one_loss = F.cross_entropy(y_pred, y_batch)
                one_loss.backward()
                optimizer.step()

                total_loss += one_loss.item()

                _, predicted = torch.max(y_pred, 1)
                correct += (predicted == y_batch).sum()
                total += len(y_batch)

                batches += 1
                # if batches % 100 == 0:
                # print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

                elapsed = pbar.format_dict['elapsed']
                elapsed_str = pbar.format_interval(elapsed)


        if len(elapsed_str) == 5:
            elapsed_str = "00:" + elapsed_str
        elapsed_str = str(datetime.datetime.strptime(elapsed_str, '%H:%M:%S').time())

        pbar.close()
        train_time.append(elapsed_str)
        total_loss = round(total_loss, 4)
        train_loss.append(total_loss)
        accuracy = round((correct.float() / total).item(), 4)
        train_accuracy.append(accuracy)
        print("Train Time",  elapsed_str, "  ", "Train Loss:", total_loss,  "  ",  "Train Accuracy:", accuracy)

        torch.save(model.state_dict(), model_save_folder + "clickbait_classifcation_model_" + str(i + 1) + ".bin")

if __name__ == '__main__':

    pretrained_model = 'klue/roberta-small'
    train_data_path = "aihub_clickbait_classification_data/train.csv"
    model_save_folder =  "./model/"
    epoch = 10
    batch_size = 64
    main(pretrained_model, train_data_path, model_save_folder, epoch, batch_size)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|          | 0/908 [00:00<?, ?it/s]/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Epoch 1: 100%|██████████| 908/908 [03:08<00:00,  4.83it/s]


Train Time 00:03:08    Train Loss: 587.3607    Train Accuracy: 0.6042


Epoch 2: 100%|██████████| 908/908 [03:08<00:00,  4.81it/s]


Train Time 00:03:08    Train Loss: 518.0104    Train Accuracy: 0.6952


...

Train Time 00:03:09    Train Loss: 191.0845    Train Accuracy: 0.9149


Epoch 9: 100%|██████████| 908/908 [03:08<00:00,  4.81it/s]


Train Time 00:03:08    Train Loss: 153.3742    Train Accuracy: 0.9338


Epoch 10: 100%|██████████| 908/908 [03:08<00:00,  4.81it/s]


Train Time 00:03:08    Train Loss: 121.9972    Train Accuracy: 0.9477


### Model Test

In [ ]:
import sys
import os
import datetime
import regex as re
import argparse
import pandas as pd
from tqdm import tqdm

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

from transformers import AutoTokenizer, RobertaForSequenceClassification


def main(pretrained_model, test_data_path, load_model_path, predction_csv_path, epoch, batch_size):

    class ClickbaitDetectionDataset(Dataset):

        def __init__(self, dataset):
            self.dataset = dataset
            self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

        def __len__(self):
            return len(self.dataset)

        def __getitem__(self, idx):
            row = self.dataset.iloc[idx, 0:1].values
            text = row[0]

            inputs = self.tokenizer(
                text,
                return_tensors='pt',
                truncation=True,
                max_length=256,
                pad_to_max_length=True,
                add_special_tokens=True
                )

            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]

            return input_ids, attention_mask

    if torch.cuda.is_available() == True:
        device = torch.device("cuda")
        model = RobertaForSequenceClassification.from_pretrained(pretrained_model).to(device)
    elif torch.cuda.is_available() == False:
        model = RobertaForSequenceClassification.from_pretrained(pretrained_model)

    checkpoint = torch.load(os.path.join(load_model_path, "clickbait_classifcation_model_" + str(epoch) + ".bin"))
    model.load_state_dict(checkpoint)
    model.eval()

    test_data = pd.read_csv(test_data_path)
    test_dataset = ClickbaitDetectionDataset(test_data)

    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    predicted_list = []

    with tqdm(test_loader) as pbar:
        for input_ids_batch, attention_masks_batch in pbar:

            if torch.cuda.is_available() == True:
                y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]

            elif torch.cuda.is_available() == False:
                y_pred = model(input_ids_batch, attention_mask=attention_masks_batch)[0]

            _, predicted = torch.max(y_pred, 1)

            for prediction in predicted.tolist():
                predicted_list.append(prediction)

            elapsed = pbar.format_dict['elapsed']
            elapsed_str = pbar.format_interval(elapsed)


        if len(elapsed_str) == 5:
            elapsed_str = "00:" + elapsed_str
        elapsed_str = str(datetime.datetime.strptime(elapsed_str, '%H:%M:%S').time())

        pbar.close()
        print("Test Time",  elapsed_str)

    y_pred_csv = pd.DataFrame({"Prediction": predicted_list})
    y_pred_csv.to_csv(predction_csv_path, index=False)

if __name__ == '__main__':

    pretrained_model = 'klue/roberta-small'
    test_data_path = "aihub_clickbait_classification_data/test_x.csv"
    load_model_path =  "./model/"
    predction_csv_path = "log/pred_y.csv"
    epoch = 5
    batch_size = 64
    main(pretrained_model, test_data_path, load_model_path, predction_csv_path, epoch, batch_size)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

Test Time 00:00:01


### Gradio

In [ ]:
!pip install gradio
!pip install --upgrade typing-extensions

In [ ]:
import gradio as gr
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification


def main(text):

    class ClickbaitDetectionDataset(Dataset):

        def __init__(self, dataset):
            self.dataset = dataset
            self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

        def __len__(self):
            return len(self.dataset)

        def __getitem__(self, idx):
            row = self.dataset.iloc[idx, 0:2].values
            text = row[0]
            y = row[1]

            inputs = self.tokenizer(
                text,
                return_tensors='pt',
                truncation=True,
                max_length=256,
                pad_to_max_length=True,
                add_special_tokens=True
                )

            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]

            return input_ids, attention_mask, y

    device = torch.device("cuda")
    model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base").to(device)
    checkpoint = torch.load("model/clickbait_classifcation_model_5.bin")
    model.load_state_dict(checkpoint)
    model.eval()

    test_data = pd.DataFrame({"Text":[text]})
    test_dataset = ClickbaitDetectionDataset(test_data)

    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

    with torch.no_grad():
        for input_ids_batch, attention_masks_batch in test_loader:
            # optimizer.zero_grad()

            y_batch = y_batch.to(device)
            y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
            _, predicted = torch.max(y_pred, 1)

            if predicted.item() == 0:
                classfication_result = "It's Not Clickbait Article"
            elif predicted.item() == 1:
                classfication_result =  "It's Clickbait Article"

    return classfication_result


if __name__ == '__main__':
    demo = gr.Interface(fn=main, inputs="text", outputs="text")
    demo.launch()
    # demo.launch( share = True , debug = True)